# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [17]:
import sklearn
sklearn.__version__

'0.21.1'

In [18]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Name'] = df['Name'].fillna('None')
mean_df = df.groupby(['Name'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Name'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Name'])['Fare'].median().reset_index()
max_df = df.groupby(['Name'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Name'])
temp = pd.merge(temp, median_df, how='left', on=['Name'])
temp = pd.merge(temp, max_df, how='left', on=['Name'])
temp.columns = ['Name', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']
temp.head(10)

,Name,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,"Abbing, Mr. Anthony",7.5500,7.5500,7.5500,7.5500
1,"Abbott, Mr. Rossmore Edward",20.2500,20.2500,20.2500,20.2500
2,"Abbott, Mrs. Stanton (Rosa Hunt)",20.2500,20.2500,20.2500,20.2500
3,"Abelson, Mr. Samuel",24.0000,24.0000,24.0000,24.0000
4,"Abelson, Mrs. Samuel (Hannah Wizosky)",24.0000,24.0000,24.0000,24.0000
5,"Adahl, Mr. Mauritz Nils Martin",7.2500,7.2500,7.2500,7.2500
6,"Adams, Mr. John",8.0500,8.0500,8.0500,8.0500
7,"Ahlin, Mrs. Johan (Johanna Persdotter Larsson)",9.4750,9.4750,9.4750,9.4750
8,"Aks, Mrs. Sam (Leah Rosen)",9.3500,9.3500,9.3500,9.3500
9,"Albimona, Mr. Nassef Cassem",18.7875,18.7875,18.7875,18.7875


In [20]:
df = pd.merge(df, temp, how='left', on=['Name'])
df = df.drop(['Name'] , axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,7.2500,7.2500,7.2500,7.2500
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,71.2833,71.2833,71.2833,71.2833
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,7.9250,7.9250,7.9250,7.9250
3,1,female,35.0,1,0,113803,53.1000,C123,S,53.1000,53.1000,53.1000,53.1000
4,3,male,35.0,0,0,373450,8.0500,NaN,S,8.0500,8.0500,8.0500,8.0500


In [21]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']



,Pclass,Age,SibSp,Parch,Fare,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,3,22.0,1,0,7.2500,7.2500,7.2500,7.2500,7.2500
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,7.9250,7.9250,7.9250,7.9250
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,8.0500,8.0500,8.0500,8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [23]:
# 原始特徵 + 邏輯斯迴歸
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max'] , axis=1)


train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [24]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7005053927832138